# Quadratically Constrained Quadratic Programs
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

- Introduce Quadratic Programs
    - Mathematical model (incl. constraints) + variable types<br>
    https://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program
    - How to build it with docplex (brief example)
    - How to build the same with the QuadraticProgram
- Example
  - How to make an optimization model for some problems

The optimization stack of Qiskit Aqua is designed after CPLEX API. The base stack `QuadraticProgram` provides almost same API as CPLEX Python API
and it can load a model from a Docplex model. 

Part of `QuadraticProgram` uses CPLEX API, for example, `from_cplex` and `write_as_string`. You need to install `cplex` package beforehand.
There are two types of `cplex` package: the [community edition](https://pypi.org/project/cplex/) and the [commercial edition](https://www.ibm.com/analytics/cplex-optimizer).
You can install the community edition by `pip install cplex`; but it requires Python 3.6 or 3.7.
On the other hand, here is the [instruction to install the commercial edition](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.10.0/ilog.odms.cplex.help/CPLEX/GettingStarted/topics/set_up/setup_overview.html).

`QuadraticProgram` is the core of making an optimization problem. It deals with quadratically constrained quadratic program as follows.
$$
\begin{align}
\text{minimize}\quad& \frac{1}{2} x^\top Q x + c^\top x\\
\text{subject to}\quad& A x \leq b\\
& x^\top Q_i x + a_i^\top x \leq r_i, \quad 1,\dots,i,\dots,q\\
& l_i \leq x_i \leq u_i, \quad 1,\dots,i,\dots,q,
\end{align}
$$
where $Q$ of the objective function is a symmetric matrix.

As setup, you need to import the following module.

In [61]:
from qiskit.optimization import OptimizationProblem as QuadraticProgram
from cplex import SparsePair, SparseTriple

You can start with a empty model or load a problem from files of particular formats such as LP or MPS.
You can set a problem name as you want.

In [35]:
mod = QuadraticProgram()
mod.set_problem_name('my problem')

There are three types of variables supported by Aqua.
- Binary varible
- Integer variable
- Continuous variable

You can specify nemes, types, lower bounds and upper bounds. `write_as_string` is a method to display the model.
`Binaries` denotes binary variables and `Generals` denotes integer variables.
If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones.

Note that you cannot use 'e' or 'E' as the first character of names due to the [specification of LP format](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.7.1/ilog.odms.cplex.help/CPLEX/FileFormats/topics/LP_VariableNames.html).

In [36]:
# Add variables
mod.variables.add(names=['x', 'y', 'z'], types='BIC', lb=[0,-1,-1], ub=[1,5,5])
print(mod.write_as_string())

Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj1: 0 x + 0 y + 0 z
Bounds
 0 <= x <= 1
-1 <= y <= 5
-1 <= z <= 5
Binaries
 x 
Generals
 y 
End



You can add either linear or quadratic objective function.

In [37]:
# Add objective function

mod.objective.set_name('obj')
mod.objective.set_linear([('x', 1), ('y', -1), ('z', 10)])
mod.objective.set_quadratic_coefficients([('x', 'x', 2), ('y', 'z', -1)])
print(mod.write_as_string())

Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x - y + 10 z + [ 2 x ^2 - 2 y * z ] / 2
Bounds
 0 <= x <= 1
-1 <= y <= 5
-1 <= z <= 5
Binaries
 x 
Generals
 y 
End



You can add linear constraints

In [62]:
# Add linear constraints
mod.linear_constraints.delete()
mod.linear_constraints.add(names=['lin_eq', 'lin_leq', 'lin_geq'], lin_expr=[SparsePair(['x', 'y'], [1, 2]), SparsePair(['x', 'y'], [1, 2]), SparsePair(['x', 'y'], [1, 2])], senses='ELG', rhs=[1,1,1])
print(mod.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x - y + 10 z + [ 2 x ^2 - 2 y * z ] / 2
Subject To
 lin_eq:  x + 2 y  = 1
 lin_leq: x + 2 y <= 1
 lin_geq: x + 2 y >= 1
 quad_eq:  x + y + [ x ^2 - y * z ]  = 1
 quad_leq: x + y + [ x ^2 - y * z ] <= 1
 quad_geq: x + y + [ x ^2 - y * z ] >= 1
Bounds
 0 <= x <= 1
-1 <= y <= 5
-1 <= z <= 5
Binaries
 x 
Generals
 y 
End



You can add quadratic constraints. Compared with linear constraints, you can add quadratic constraints one by one.

In [63]:
# Add quadratic constraints
mod.quadratic_constraints.delete()
mod.quadratic_constraints.add(name='quad_eq', lin_expr=SparsePair(['x', 'y'], [1, 1]), quad_expr=SparseTriple(['x', 'y'], ['x', 'z'], [1, -1]), sense='E', rhs=1)
mod.quadratic_constraints.add(name='quad_leq', lin_expr=SparsePair(['x', 'y'], [1, 1]), quad_expr=SparseTriple(['x', 'y'], ['x', 'z'], [1, -1]), sense='L', rhs=1)
mod.quadratic_constraints.add(name='quad_geq', lin_expr=SparsePair(['x', 'y'], [1, 1]), quad_expr=SparseTriple(['x', 'y'], ['x', 'z'], [1, -1]), sense='G', rhs=1)
print(mod.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj: x - y + 10 z + [ 2 x ^2 - 2 y * z ] / 2
Subject To
 lin_eq:  x + 2 y  = 1
 lin_leq: x + 2 y <= 1
 lin_geq: x + 2 y >= 1
 quad_eq:  x + y + [ x ^2 - y * z ]  = 1
 quad_leq: x + y + [ x ^2 - y * z ] <= 1
 quad_geq: x + y + [ x ^2 - y * z ] >= 1
Bounds
 0 <= x <= 1
-1 <= y <= 5
-1 <= z <= 5
Binaries
 x 
Generals
 y 
End



You can find detailed information at [Qiskit Aqua API references](https://qiskit.org/documentation/apidoc/optimization/optimization.html) and [CPLEX Python Reference Manual](https://www.ibm.com/support/knowledgecenter/SSSA5P_12.10.0/ilog.odms.cplex.help/refpythoncplex/html/help.html).

You can substitue some of variables with constants or other variables.
More precicely, `QuadraticProgram` has a method `substitute_variables(constants=..., variables=...)` to deal with the following two cases.
- $x \leftarrow c$: when `constants` have a list of `SparsePair`, `SparsePair.ind` and `SparsePair.val` correspond to $x$ and $c$, respectively. 
- $x \leftarrow c y$: when `variables` have a list of `SparseTriples`, `SparseTriple.ind1`, `SparseTriple.ind2`, and `SparseTriple.val` correspond to $x$, $y$, and $c$, respectively.


In [67]:
# substitute variables
sub, status = mod.substitute_variables(constants=SparsePair(['x'], [0]), variables=SparseTriple(['y'], ['z'], [0.5]))
print('status', status)
print(sub.write_as_string())

status SubstitutionStatus.success
\ENCODING=ISO-8859-1
\Problem name: my problem

Minimize
 obj1: 9.5 z + [ - z ^2 ] / 2
Subject To
 lin_eq:  z  = 1
 lin_leq: z <= 1
 lin_geq: z >= 1
 quad_eq:  0.5 z + [ - 0.5 z ^2 ]  = 1
 quad_leq: 0.5 z + [ - 0.5 z ^2 ] <= 1
 quad_geq: 0.5 z + [ - 0.5 z ^2 ] >= 1
Bounds
-1 <= z <= 5
End



If the resulting problem is infeasible due to lower bounds or upper bounds, the methods returns the statsu `SubstitutionStatus.infeasible`.

In [66]:
sub, status = mod.substitute_variables(constants=SparsePair(ind=['x'], val=[-1]))
print('status', status)

Infeasible substitution for variable: x


status SubstitutionStatus.infeasible
